In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/SD_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3507637205,SD,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,4.612702e+14
1,1221237209,SD,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,4.610301e+14
2,4673147200,SD,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,4.600596e+14
3,6663527703,SD,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,4.609901e+14
4,7971747009,SD,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,4.608301e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3507637205,SD,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,4.612702e+14
1,1221237209,SD,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,4.610301e+14
2,4673147200,SD,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,4.600596e+14
3,6663527703,SD,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,4.609901e+14
4,7971747009,SD,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,4.608301e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
389,5274317004,SD,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,NaN
660,6881127209,SD,"2601 S MINNESOTA AVE # 105-249, SIOUX FALLS, SD",43.521107,-96.731963,NaN
1301,9903297002,SD,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('46')] #SD
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/3 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

3


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15072/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
389,5274317004,SD,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,[]
660,6881127209,SD,"2601 S MINNESOTA AVE # 105-249, SIOUX FALLS, SD",43.521107,-96.731963,[460990015012015]
1301,9903297002,SD,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15072/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
389,5274317004,SD,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,NaN
660,6881127209,SD,"2601 S MINNESOTA AVE # 105-249, SIOUX FALLS, SD",43.521107,-96.731963,460990015012015
1301,9903297002,SD,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
389,5274317004,SD,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,NaN
1301,9903297002,SD,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
389,5274317004,NaN
660,6881127209,460990015012015
1301,9903297002,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/SD_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,789418,3507637205,SD,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,4.612702e+14
1,789419,1221237209,SD,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,4.610301e+14
2,789420,4673147200,SD,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,4.600596e+14
3,789421,6663527703,SD,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,4.609901e+14
4,789422,7971747009,SD,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,4.608301e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,789418,3507637205,SD,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,4.612702e+14,NaN
1,789419,1221237209,SD,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,4.610301e+14,NaN
2,789420,4673147200,SD,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,4.600596e+14,NaN
3,789421,6663527703,SD,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,4.609901e+14,NaN
4,789422,7971747009,SD,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,4.608301e+14,NaN
...,...,...,...,...,...,...,...,...
2690,792108,8914837010,SD,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,4.608301e+14,NaN
2691,792109,9178718708,SD,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,4.600596e+14,NaN
2692,792110,9304517103,SD,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,4.610301e+14,NaN
2693,792111,9378667203,SD,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,4.609900e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,789418,3507637205,SD,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,4.612702e+14,NaN,461270203011033.0
1,789419,1221237209,SD,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,4.610301e+14,NaN,461030114003026.0
2,789420,4673147200,SD,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,4.600596e+14,NaN,460059568001132.0
3,789421,6663527703,SD,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,4.609901e+14,NaN,460990104051004.0
4,789422,7971747009,SD,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,4.608301e+14,NaN,460830101073026.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3507637205,SD,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,461270203011033.0
1,1221237209,SD,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,461030114003026.0
2,4673147200,SD,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,460059568001132.0
3,6663527703,SD,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,460990104051004.0
4,7971747009,SD,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,460830101073026.0
...,...,...,...,...,...,...
2690,8914837010,SD,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,460830101162011.0
2691,9178718708,SD,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,460059568002064.0
2692,9304517103,SD,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,461030110052001.0
2693,9378667203,SD,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,460990015013029.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
389,5274317004,SD,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,NaN
1301,9903297002,SD,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,NaN


Convert to File

In [19]:
fips_SD = fips_merge_drop

In [20]:
fips_SD.to_csv("../../../data/state_data/geo/geo_fips/23/SD_fips_scraped.csv") 